In [1]:

import os
import sys

sys.path.append(
    os.path.abspath("./codebase")
)
import torch.backends.cudnn as cudnn
import random
import time
from collections import OrderedDict
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import one_hot
import sklearn.metrics as metrics
import utils
from Explainer.loss_F import loss_fn_kd, entropy_loss
from Explainer.models.Gated_Logic_Net import Gated_Logic_Net
from Explainer.models.explainer import Explainer
from Explainer.models.pi import Pi
from dataset.dataset_cubs import Dataset_cub_for_explainer
from dataset.utils_dataset import get_dataset_with_image_and_attributes
from Explainer.loss_F import loss_fn_kd, entropy_loss, Selective_Distillation_Loss
from Explainer.models.concepts import Conceptizator

import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'torch'

In [5]:
lr = 0.01
cov = 0.2
pickle_in = open(
    os.path.join(
        f"./out/cub/explainer/ResNet101/lr_{lr}_epochs_500_temperature-lens_0.7_use-concepts-as-pi-input_True_input-size-pi_2048_cov_{cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1_layer_layer4_explainer_init_none",
        "cov_0.2_lr_0.01",
        "iter8",
        "explainer",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr)
print(args.cov)

FileNotFoundError: [Errno 2] No such file or directory: './out/cub/explainer/ResNet101/lr_0.01_epochs_500_temperature-lens_0.7_use-concepts-as-pi-input_True_input-size-pi_2048_cov_0.2_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1_layer_layer4_explainer_init_none/cov_0.2_lr_0.01/iter8/explainer/test_explainer_configs.pkl'

In [6]:
lr = 0.01
cov = 0.2
pickle_in = open(
    os.path.join(
        f"./out/cub/explainer/ResNet101/lr_{lr}_epochs_500_temperature-lens_0.7_use-concepts-as-pi-input_True_input-size-pi_2048_cov_{cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1_layer_layer4_explainer_init_none",
        "iter1",
        "explainer",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr)
print(args.cov)

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_use-concepts-as-pi-input_{use_concepts_as_pi_input}_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
    f"_layer_{args.layer}_explainer_init_{explainer_init if not args.explainer_init else args.explainer_init}"
)

root = "./out/cub/"
experiment = f"explainer/ResNet101/{experiment_folder}"
iteration = "iter1"
expert_type = "explainer"
output = "g_outputs"


train_tensor_image_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "train_tensor_images.pt"
    )
)

train_tensor_y_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "train_tensor_y.pt"
    )
)

train_concept_mask_1 = torch.load(
    os.path.join(root, experiment, iteration, expert_type, output, "train_mask_alpha.pt")
)

test_tensor_image_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "test_tensor_images.pt"
    )
)

test_tensor_y_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "test_tensor_y.pt"
    )
)

test_concept_mask_1 = torch.load(
    os.path.join(root, experiment, iteration, expert_type, output, "test_mask_alpha.pt")
)


print("\n\n << Train sizes >>")
print(train_tensor_image_1.size())
print(train_tensor_y_1.size())
print(train_concept_mask_1.size())


print("\n\n << Test sizes >>")
print(test_tensor_image_1.size())
print(test_tensor_y_1.size())
print(test_concept_mask_1.size())


FileNotFoundError: [Errno 2] No such file or directory: './out/cub/explainer/ResNet101/lr_0.01_epochs_500_temperature-lens_0.7_use-concepts-as-pi-input_True_input-size-pi_2048_cov_0.2_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1_layer_layer4_explainer_init_none/iter1/explainer/test_explainer_configs.pkl'

In [4]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"./out/cub/explainer/ResNet101/lr_{base_lr}_epochs_500_temperature-lens_0.7_use-concepts-as-pi-input_True_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1_layer_layer4_explainer_init_none",
        "cov_0.2_lr_0.01",
        "iter2",
        "explainer",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_use-concepts-as-pi-input_{use_concepts_as_pi_input}_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
    f"_layer_{args.layer}_explainer_init_{explainer_init if not args.explainer_init else args.explainer_init}"
)

root = "./out/cub/"
experiment = f"explainer/ResNet101/{experiment_folder}"
iteration = "iter2"
expert_type = "explainer"
output = "g_outputs"


train_tensor_images_2 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_2 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_mask_alpha_2 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)


test_tensor_images_2 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_images.pt",
    )
)

test_tensor_y_2 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_y.pt",
    )
)

test_mask_alpha_2 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_images_2.size())
print(train_tensor_y_2.size())
print(train_mask_alpha_2.size())

print("\n\n << Test sizes >>")
print(test_tensor_images_2.size())
print(test_tensor_y_2.size())
print(test_mask_alpha_2.size())


########################
0.01
0.2


 << Train sizes >>
torch.Size([1209, 3, 448, 448])
torch.Size([1209])
torch.Size([1209, 108])


 << Test sizes >>
torch.Size([133, 3, 448, 448])
torch.Size([133])
torch.Size([133, 108])


<IPython.core.display.Javascript object>

In [5]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"./out/cub/explainer/ResNet101/lr_{base_lr}_epochs_500_temperature-lens_0.7_use-concepts-as-pi-input_True_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1_layer_layer4_explainer_init_none",
        "cov_0.2_lr_0.01",
        "iter3",
        "explainer",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_use-concepts-as-pi-input_{use_concepts_as_pi_input}_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
    f"_layer_{args.layer}_explainer_init_{explainer_init if not args.explainer_init else args.explainer_init}"
)

root = "./out/cub/"
experiment = f"explainer/ResNet101/{experiment_folder}"
iteration = "iter3"
expert_type = "explainer"
output = "g_outputs"


train_tensor_images_3 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_3 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_mask_alpha_3 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)


test_tensor_images_3 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_images.pt",
    )
)

test_tensor_y_3 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_y.pt",
    )
)

test_mask_alpha_3 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_images_3.size())
print(train_tensor_y_3.size())
print(train_mask_alpha_3.size())

print("\n\n << Test sizes >>")
print(test_tensor_images_3.size())
print(test_tensor_y_3.size())
print(test_mask_alpha_3.size())


########################
0.01
0.2


 << Train sizes >>
torch.Size([1292, 3, 448, 448])
torch.Size([1292])
torch.Size([1292, 108])


 << Test sizes >>
torch.Size([153, 3, 448, 448])
torch.Size([153])
torch.Size([153, 108])


<IPython.core.display.Javascript object>

In [6]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"./out/cub/explainer/ResNet101/lr_{base_lr}_epochs_500_temperature-lens_0.7_use-concepts-as-pi-input_True_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1_layer_layer4_explainer_init_none",
        "cov_0.2_lr_0.01",
        "iter4",
        "explainer",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_use-concepts-as-pi-input_{use_concepts_as_pi_input}_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
    f"_layer_{args.layer}_explainer_init_{explainer_init if not args.explainer_init else args.explainer_init}"
)

root = "./out/cub/"
experiment = f"explainer/ResNet101/{experiment_folder}"
iteration = "iter4"
expert_type = "explainer"
output = "g_outputs"


train_tensor_images_4 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_4 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_mask_alpha_4 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)


test_tensor_images_4 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_images.pt",
    )
)

test_tensor_y_4 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_y.pt",
    )
)

test_mask_alpha_4 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_images_4.size())
print(train_tensor_y_4.size())
print(train_mask_alpha_4.size())

print("\n\n << Test sizes >>")
print(test_tensor_images_4.size())
print(test_tensor_y_4.size())
print(test_mask_alpha_4.size())


########################
0.01
0.2


 << Train sizes >>
torch.Size([1292, 3, 448, 448])
torch.Size([1292])
torch.Size([1292, 108])


 << Test sizes >>
torch.Size([176, 3, 448, 448])
torch.Size([176])
torch.Size([176, 108])


<IPython.core.display.Javascript object>

In [7]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"./out/cub/explainer/ResNet101/lr_{base_lr}_epochs_500_temperature-lens_0.7_use-concepts-as-pi-input_True_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1_layer_layer4_explainer_init_none",
        "cov_0.2_lr_0.01",
        "iter5",
        "explainer",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_use-concepts-as-pi-input_{use_concepts_as_pi_input}_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
    f"_layer_{args.layer}_explainer_init_{explainer_init if not args.explainer_init else args.explainer_init}"
)

root = "./out/cub/"
experiment = f"explainer/ResNet101/{experiment_folder}"
iteration = "iter5"
expert_type = "explainer"
output = "g_outputs"


train_tensor_images_5 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_5 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_mask_alpha_5 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)


test_tensor_images_5 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_images.pt",
    )
)

test_tensor_y_5 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_y.pt",
    )
)

test_mask_alpha_5 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_images_5.size())
print(train_tensor_y_5.size())
print(train_mask_alpha_5.size())

print("\n\n << Test sizes >>")
print(test_tensor_images_5.size())
print(test_tensor_y_5.size())
print(test_mask_alpha_5.size())


########################
0.01
0.2


 << Train sizes >>
torch.Size([1139, 3, 448, 448])
torch.Size([1139])
torch.Size([1139, 108])


 << Test sizes >>
torch.Size([146, 3, 448, 448])
torch.Size([146])
torch.Size([146, 108])


<IPython.core.display.Javascript object>

In [8]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"./out/cub/explainer/ResNet101/lr_{base_lr}_epochs_500_temperature-lens_0.7_use-concepts-as-pi-input_True_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1_layer_layer4_explainer_init_none",
        "cov_0.2_lr_0.01",
        "iter6",
        "explainer",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_use-concepts-as-pi-input_{use_concepts_as_pi_input}_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
    f"_layer_{args.layer}_explainer_init_{explainer_init if not args.explainer_init else args.explainer_init}"
)

root = "./out/cub/"
experiment = f"explainer/ResNet101/{experiment_folder}"
iteration = "iter6"
expert_type = "explainer"
output = "g_outputs"


train_tensor_images_6 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_images.pt",
    )
)

train_tensor_y_6 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_tensor_y.pt",
    )
)

train_mask_alpha_6 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "train_mask_alpha.pt",
    )
)


test_tensor_images_6 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_images.pt",
    )
)

test_tensor_y_6 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_tensor_y.pt",
    )
)

test_mask_alpha_6 = torch.load(
    os.path.join(
        root,
        experiment,
        "cov_0.2_lr_0.01",
        iteration,
        expert_type,
        output,
        "test_mask_alpha.pt",
    )
)


print("\n\n << Train sizes >>")
print(train_tensor_images_6.size())
print(train_tensor_y_6.size())
print(train_mask_alpha_6.size())

print("\n\n << Test sizes >>")
print(test_tensor_images_6.size())
print(test_tensor_y_6.size())
print(test_mask_alpha_6.size())


########################
0.01
0.2


 << Train sizes >>
torch.Size([1345, 3, 448, 448])
torch.Size([1345])
torch.Size([1345, 108])


 << Test sizes >>
torch.Size([189, 3, 448, 448])
torch.Size([189])
torch.Size([189, 108])


<IPython.core.display.Javascript object>

In [11]:
train_tensor_images = torch.cat(
    (train_tensor_image_1, train_tensor_images_2, train_tensor_images_3, train_tensor_images_4, 
     train_tensor_images_5, train_tensor_images_6), dim=0
)
train_tensor_y = torch.cat(
    (train_tensor_y_1, train_tensor_y_2, train_tensor_y_3, train_tensor_y_4,
    train_tensor_y_5, train_tensor_y_6), dim=0
)
train_mask_alpha = torch.cat(
    (train_concept_mask_1, train_mask_alpha_2, train_mask_alpha_3, train_mask_alpha_4, train_mask_alpha_5,
    train_mask_alpha_6), dim=0
)

print(train_tensor_images.size())
print(train_tensor_y.size())
print(train_mask_alpha.size())


torch.Size([8241, 3, 448, 448])
torch.Size([8241])
torch.Size([8241, 108])


<IPython.core.display.Javascript object>

In [12]:
test_tensor_images = torch.cat(
    (test_tensor_image_1, test_tensor_images_2, test_tensor_images_3, test_tensor_images_4,
    test_tensor_images_5, test_tensor_images_6), dim=0
)
test_tensor_y = torch.cat(
    (test_tensor_y_1, test_tensor_y_2, test_tensor_y_3, test_tensor_y_4, test_tensor_y_5, test_tensor_y_6), dim=0
)
test_mask_alpha = torch.cat(
    (test_concept_mask_1, test_mask_alpha_2, test_mask_alpha_3, test_mask_alpha_4, test_mask_alpha_5,
    test_mask_alpha_6), dim=0
)

print(test_tensor_images.size())
print(test_tensor_y.size())
print(test_mask_alpha.size())


torch.Size([1040, 3, 448, 448])
torch.Size([1040])
torch.Size([1040, 108])


<IPython.core.display.Javascript object>

In [13]:
save_path = f"./out/{args.dataset}/completeness/{args.arch}/dataset"
os.makedirs(save_path, exist_ok=True)
print(save_path)
torch.save(train_tensor_images, os.path.join(save_path, f"train_tensor_images.pt"))
torch.save(train_tensor_y, os.path.join(save_path, f"train_tensor_y.pt"))
torch.save(train_mask_alpha, os.path.join(save_path, f"train_mask_alpha.pt"))

torch.save(test_tensor_images, os.path.join(save_path, f"test_tensor_images.pt"))
torch.save(test_tensor_y, os.path.join(save_path, f"test_tensor_y.pt"))
torch.save(test_mask_alpha, os.path.join(save_path, f"test_mask_alpha.pt"))

./out/cub/completeness/ResNet101/dataset


<IPython.core.display.Javascript object>